In [1]:
import sys
import os

sys.path.append(os.path.abspath("../common"))

import math
import time
import numpy as np
import cv2
import pynq
import dac_sdc
import ctypes
from pynq import Clocks

team_name = 'InvolutionNet'
team = dac_sdc.Team(team_name)

In [2]:
overlay = pynq.Overlay(team.get_bitstream_path())
cfuns = ctypes.cdll.LoadLibrary("./yolo.so")
dma = overlay.axi_dma_0
nn_ctrl = overlay.ultra_net_0

In [3]:
BATCH_SIZE = 50
IMAGE_RAW_ROW = 360
IMAGE_RAW_COL = 640
IMAGE_ROW = 160
IMAGE_COL = 320
GRID_ROw = 10
GRID_COL = 20
X_SCALE = IMAGE_RAW_COL / IMAGE_COL
Y_SCALE = IMAGE_RAW_ROW / IMAGE_ROW


in_buffer0 = pynq.allocate(shape=(BATCH_SIZE, IMAGE_RAW_ROW, IMAGE_RAW_COL, 3), dtype=np.uint8, cacheable = 1)
in_buffer1 = pynq.allocate(shape=(BATCH_SIZE, IMAGE_RAW_ROW, IMAGE_RAW_COL, 3), dtype=np.uint8, cacheable = 1)
in_buffers = [in_buffer0, in_buffer1]
out_buffer0 = pynq.allocate(shape=(BATCH_SIZE, GRID_ROw, GRID_COL, 6, 6), dtype=np.int32, cacheable = 1)
out_buffer1 = pynq.allocate(shape=(BATCH_SIZE, GRID_ROw, GRID_COL, 6, 6), dtype=np.int32, cacheable = 1)
out_buffers = [out_buffer0, out_buffer1]

which_buffer = 0
first_batch = True
net_cnt = 0
last_batch_size = BATCH_SIZE

def yolo(out_buffer, batch_n, result):
    out_buffer_dataptr=out_buffer.ctypes.data_as(ctypes.c_char_p)
    rst=np.empty( (batch_n,4),dtype=np.int32)
    rst_dataptr=rst.ctypes.data_as(ctypes.c_char_p)
    cfuns.yolo(out_buffer_dataptr,batch_n,rst_dataptr)
    result.extend(rst.tolist())
    
        
################################compute##################################
def compute_stream(rgb_imgs):
    rgblen = len(rgb_imgs)
    image_paths = []
    img_locations = list()
    global first_batch
    global which_buffer    
    global net_cnt
    global last_batch_size
    global last_status
    
    while True:
        if len(rgb_imgs) is 0:
            break
        bcount = 0
        initflag = True
        while True:
            bstart = bcount*BATCH_SIZE
            bcount += 1
            bend = bcount*BATCH_SIZE
            
            if initflag == True:
                initflag = False
                which_buffer = 0
                first_batch = True
                net_cnt = 0
                last_batch_size = BATCH_SIZE
                
            if first_batch == True:                             #copy image data to contiguous buffer
                first_batch = False
                which_buffer = 0
                #bsize = Pload2buff(which_buffer,bstart,bend,rgblen)
                lcount = 0
                if bstart >= rgblen:
                    lcount = 0
                elif bend > rgblen:
                    for (img_path, img) in rgb_imgs[bstart:]:
                        in_buffers[which_buffer][lcount][:] = img
                        lcount += 1
                    for i in range(BATCH_SIZE - lcount):
                        in_buffers[which_buffer][lcount+i][:] = in_buffers[which_buffer][0][:]
                else:
                    for (img_path, img) in rgb_imgs[bstart:bend]:
                        in_buffers[which_buffer][lcount][:] = img
                        lcount += 1
                bsize = lcount
                if bsize >0 and bsize < BATCH_SIZE:
                    last_batch_size = bsize
            else:
                # count
                net_cnt += 1
                nn_ctrl.write(0x10, in_buffers[which_buffer].shape[0])
                nn_ctrl.write(0x0, 1) # Deassert reset
                dma.recvchannel.transfer(out_buffers[which_buffer])
                dma.sendchannel.transfer(in_buffers[which_buffer])
                
                # switch buffer
                if which_buffer == 0:
                    which_buffer = 1
                else:
                    which_buffer = 0
                    
                #bsize = Pload2buff(which_buffer,bstart,bend,rgblen)
                lcount = 0
                if bstart >= rgblen:
                    lcount = 0
                elif bend > rgblen:
                    for (img_path, img) in rgb_imgs[bstart:]:
                        in_buffers[which_buffer][lcount][:] = img
                        lcount += 1
                    for i in range(BATCH_SIZE - lcount):
                        in_buffers[which_buffer][lcount+i][:] = in_buffers[which_buffer][0][:]
                else:
                    for (img_path, img) in rgb_imgs[bstart:bend]:
                        in_buffers[which_buffer][lcount][:] = img
                        lcount += 1
                bsize = lcount
                if bsize >0 and bsize < BATCH_SIZE:
                    last_batch_size = bsize
                
                if net_cnt > 1:
                    yolo(out_buffers[which_buffer], BATCH_SIZE, img_locations)
                    
                dma.sendchannel.wait()
                dma.recvchannel.wait()
                
                if bsize == 0:
                    yolo(out_buffers[(which_buffer + 1) % 2], last_batch_size, img_locations)
            
            if bsize == 0:
                break
        break
    
    return img_locations

In [4]:
team.run(compute_stream, debug=True)

del in_buffer0
del in_buffer1
del in_buffers
del out_buffer0
del out_buffer1
del out_buffers

Batch 1 starting. 1000 images.
Batch 1 done. Runtime = 2.8964085578918457 seconds. Energy = 5.893978444092415 J.
Done all batches. Total runtime = 2.8964085578918457 seconds. Total energy = 5.893978444092415 J.
Savings results to XML...
XML results written successfully.
